## Setting up 

In [1]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, LabelSet
import numpy as np

output_notebook()

Loading BokehJS ...

## Defining Variables

In [2]:
# long string variables
twentysix = [1,2,3,4,5,6,7,8,9,10,11,12,
             13,14,15,16,17,18,19,20,21,22,23,24,25,26]
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
            'n','o','p','q','r','s','t','u','v','w','x','y','z']
alphabet = alphabet*2


# aesthetic variables
x_offset = -4.5    # letter annotation offset from circle edge
y_offset = -7    # letter annotation offset from circle edge

seg_color = 'gold'    # segment color
line_width = 3.0    # segment line width
seg_alpha = 0.6    # segment alpha

circ_color = 'gold'    # circle color
circ_size = 23    # circle size
circ_alpha = 0.4    # circle alpha
hov_color = 'gold'    # hover color for circles
hov_alpha = 0.8    # hover alpha for circles

font = 'Garamond'


# some points
x = np.array([15,20])
x = np.repeat(x,26)
y = twentysix*2

## Creating X and Y values for paths

In [3]:
# X values
xvals = [[15,20]]*26

In [4]:
# Y values
# rotor 3
yvals = [[1,2],[2,4],[3,6],[4,8],[5,10],[6,12],[7,3],[8,16],[9,18],
    [10,20],[11,24],[12,22],[13,26],[14,14],[15,25],[16,5],[17,9],
    [18,23],[19,7],[20,1],[21,11],[22,13],[23,21],[24,19],[25,17],
    [26,15]]

# Rotor Links - Rotor 3 [elephant]

In [5]:
# establish links
links = {
    0:[27],     1:[29],      2:[31],     3:[33],     4:[35],     5:[37],      
    6:[28],     7:[41],      8:[43],     9:[45],    10:[49],    11:[47],   
   12:[51],    13:[39],     14:[50],    15:[30],    16:[34],    17:[48],    
   18:[32],    19:[26],     20:[36],    21:[38],   22:[46],     23:[44],    
   24:[42],    25:[40], 
        
    26:[19],    27:[0],    28:[6],    29:[1],    30:[15],    31:[2],
    32:[18],    33:[3],    34:[16],   35:[4],    36:[20],    37:[5],
    38:[21],    39:[13],   40:[25],   41:[7],    42:[24],    43:[8],
    44:[23],    45:[9],    46:[22],   47:[11],   48:[17],    49:[10],  
    50:[14],    51:[12]
}

## Create Graph and Add Features

In [6]:
# create graph
p = figure(width=400, height=800, tools="",
           toolbar_location=None,title='Rotor 3')

# add letter labels
lsource = ColumnDataSource(data=dict(x = x, y = y, names = alphabet))
labels = LabelSet(x='x', y='y', text='names', level='annotation',
                  x_offset=x_offset, y_offset=y_offset, source=lsource,
                  render_mode='canvas', text_font=font, 
                  text_color='black', text_font_style='bold')
p.add_layout(labels)

# add hover glyphs
source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1':[]})
sr = p.segment(x0='x0',y0='y0',x1='x1', y1='y1', color=seg_color, 
               alpha=seg_alpha, line_width=line_width, source=source,)
cr = p.circle(x, y, color=circ_color, size=circ_size, alpha=circ_alpha, 
              hover_color=hov_color, hover_alpha=hov_alpha)

# add path glyphs
p.multi_line(xvals,yvals,color='gray',alpha=0.3)

# remove gridlines and axis labels
p.xgrid.visible = False
p.ygrid.visible = False
p.axis.visible = False

# JS for hover tool
code = """
var links = %s;
var data = {'x0': [], 'y0': [], 'x1': [], 'y1': []};
var cdata = circle.get('data');
var indices = cb_data.index['1d'].indices;
for (i=0; i < indices.length; i++) {
    ind0 = indices[i]
    for (j=0; j < links[ind0].length; j++) {
        ind1 = links[ind0][j];
        data['x0'].push(cdata.x[ind0]);
        data['y0'].push(cdata.y[ind0]);
        data['x1'].push(cdata.x[ind1]);
        data['y1'].push(cdata.y[ind1]);
    }
}
segment.set('data', data);
""" % links

# Call JS and Add HoverTool
callback = CustomJS(args={'circle':cr.data_source, 
                          'segment':sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None,callback=callback,renderers=[cr]))

# Display Graph

In [7]:
show(p)